In [1]:
# wrangling
import numpy as np
import pandas as pd
import feather
pd.options.display.max_columns = 200

# preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit

# model
from keras.optimizers import Adam
from keras.models import Sequential  
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping
import tensorflow as tf
from keras import backend as K

# metrics
from sklearn.metrics import accuracy_score, log_loss

# others
import os
import random as rn
from datetime import datetime
from tqdm import tqdm_notebook as tqdm
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


# load data

In [2]:
def load_rnn_data(path, window, predict_ts, isdim3=True, geo_col=["geoid10_tract"], y_cols=["crime"]):
    """
    y_cols: ["crime"] or ["incident_type_0", "incident_type_1", "incident_type_2"]
    geo_col: ["geoid10_tract"] or ["geoid10_block"]
    return y_all and x_all of given path
    """
    # load data
    df = feather.read_dataframe(path)
    df.sort_values(by=["datetime", "geoid10_tract"], inplace=True)
    df.set_index("datetime", inplace=True)

    # input columns
    x_cols = list(df.drop(y_cols + geo_col, axis=1).columns)

    # group by geoid
    geo_grs = df.groupby(by=geo_col)

    # arrayes to store x and y
    # (no of timesteps, window size, no of tracts,  no of features, )
    n_timesteps = int(len(df) / len(geo_grs)) - window - predict_ts + 1
    x_all = np.empty(shape=(n_timesteps, window, len(geo_grs), len(x_cols + y_cols)))

    # (output size, no of tracts, no of outputs)
    y_all = np.empty(shape=(n_timesteps, len(geo_grs), len(y_cols)))

    # to store geo_ids and y_all's datetime
    geo_ids = []

    y_datetime = df.index.unique()[window + predict_ts - 1:]

    for i, (geo_id, gr) in enumerate(tqdm(geo_grs)):
        geo_ids.append(geo_id)
        x_values = gr[y_cols + x_cols].values
        y_values = gr[y_cols].values

        for j in range(window, len(gr) - predict_ts + 1):
            # generate x_all
            x_all[j - window, :, i, :] = x_values[j - window:j, :]
            y_all[j - window, i, :] = y_values[j + predict_ts - 1, :]

    if isdim3:
        x_all = np.reshape(x_all,
                           newshape=(x_all.shape[0], x_all.shape[1], x_all.shape[2] * x_all.shape[3]))
        y_all = np.reshape(y_all,
                           newshape=(y_all.shape[0], y_all.shape[1] * y_all.shape[2]))

    return x_all, y_all, geo_ids, y_datetime

In [3]:
# set configuration
path = "./features/features_binary_tract_2H.feather"
window = 12
predict_ts = 1  # how many timesteps future does the model predict? 

In [4]:
# load x and y
x_all, y_all, geo_ids, y_datetime = load_rnn_data(path=path,
                                                  window=window,
                                                  predict_ts=predict_ts,
                                                  isdim3=True,
                                                  geo_col=["geoid10_tract"],
                                                  y_cols=["crime"])

In [5]:
print(x_all.shape)
print(y_all.shape)
print(len(geo_ids))  # to convert model output later
print(len(y_datetime))  # to convert model output later

(18600, 12, 1560)
(18600, 195)
195
18600


# preprocessing

In [6]:
# # scaling
# scaler = MinMaxScaler()
# x_all = scaler.fit_transform(x_all)

# modeling

In [7]:
def time_series_cv(x_all, y_all, n_splits=5, model=None, fit_params=None, baseline=False):
    """
    :param baseline: True or False (defualt: False)
    :return: train and test scores and prediction of y on test data
    """

    # prepare dictionary to store scores
    train_scores = {}
    metrics = ["acc", "log_loss"]
    for metric in metrics:
        train_scores[metric] = []
    test_scores = deepcopy(train_scores)

    # prepare dictionary to store predictions
    y_test_probs = np.zeros_like(y_all)

    # time series split
    tss = TimeSeriesSplit(n_splits=n_splits)

    for split, (train_idx, test_idx) in enumerate(tss.split(x_all, y_all)):

        print("---------- split {0} ----------".format(split))
        print("[{0:%H:%M:%S}] train_index:{1}~{2} test_index:{3}~{4}".format(
            datetime.now(), train_idx[0], train_idx[-1], test_idx[0], test_idx[-1]))

        # create train and test set
        x_train = x_all[:train_idx[-1]]
        y_train = y_all[:train_idx[-1]]
        x_test = x_all[test_idx[0]:test_idx[-1]]
        y_test = y_all[test_idx[0]:test_idx[-1]]

        if baseline:
            # return 0 for all predicted probabiliby
            y_train_prob = np.zeros_like(y_train)
            y_test_prob = np.zeros_like(y_test)

            # return 0 for all binary predictions
            y_train_pred = np.zeros_like(y_train)
            y_test_pred = np.zeros_like(y_test)

        else:            
            # train
            model.fit(x_train, y_train, **fit_params)

            # predict
            y_train_prob = model.predict(x_train)
            y_test_prob = model.predict(x_test)

            # convert form probability to binary
            y_train_pred = np.around(y_train_prob)
            y_test_pred = np.around(y_test_prob)

        # store test prediction
        y_test_probs[test_idx[0]:test_idx[-1]] = y_test_prob

        # calculate metrics
        train_log_loss = log_loss(y_train.flatten(), y_train_prob.flatten())
        test_log_loss = log_loss(y_test.flatten(), y_test_prob.flatten())
        train_acc = accuracy_score(y_train.flatten(), y_train_pred.flatten())
        test_acc = accuracy_score(y_test.flatten(), y_test_pred.flatten())

        # store scores
        train_scores["log_loss"].append(train_log_loss)
        test_scores["log_loss"].append(test_log_loss)
        train_scores["acc"].append(train_acc)
        test_scores["acc"].append(test_acc)

        print("[{0:%H:%M:%S}] train_log_loss:{1} test_log_loss:{2}".format(
            datetime.now(), train_log_loss, test_log_loss))
        print("[{0:%H:%M:%S}] train_acc:{1} test_acc:{2}\n".format(
            datetime.now(), train_acc, test_acc))

        # convert to dataframe
        train_scores_df = pd.DataFrame(train_scores)
        test_scores_df = pd.DataFrame(test_scores)

    return train_scores_df, test_scores_df, y_test_probs

In [8]:
class MyLSTM():
    """
    class to input into function time_series_cv
    """
    def __init__(self, units=100,
                 dropout_rate=0.2, activation="sigmoid",
                 optimizer="adam", loss="binary_crossentropy"):
        """
        define LSTM model with given arguments
        """
        self.units = units
        self.dropout_rate = dropout_rate
        self.activation = activation
        self.optimizer = optimizer
        self.loss = loss

    def fit(self, x_train, y_train, epochs=200, early_stopping_patience=5,
            batch_size=128, validation_split=0.1, train_shuffle=False):
        """
        fit train data
        """
        self.model = Sequential()
        self.model.add(LSTM(units=self.units, input_shape=(x_train.shape[1], x_train.shape[2])))
        self.model.add(Dropout(self.dropout_rate))
        self.model.add(Dense(units=y_train.shape[1], activation=self.activation))
        self.model.compile(optimizer=self.optimizer, loss=self.loss, metrics=["acc"])

        earlystopping = EarlyStopping(monitor="val_loss", patience=early_stopping_patience)
        self.model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
                       validation_split=validation_split, shuffle=train_shuffle,
                       callbacks=[earlystopping])
        
    def predict(self, x_test):
        """
        return predicted values of x_test
        """
        y_prob = self.model.predict(x_test)
        return y_prob

In [9]:
# parameters at model definition
def_params = {
    "units":100,
    "dropout_rate":0.159961594635,
    "activation":"sigmoid",
    "optimizer":Adam(lr=1.77547438185e-05),
    "loss":"binary_crossentropy"
}

# parameters at model training
fit_params = {
    "epochs":500,
    "early_stopping_patience":5,
    "batch_size":128,
    "validation_split":0.1,
    "train_shuffle":False
}

# number of time series cv splits
n_splits = 5

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
# set seed
np.random.seed(0)
os.environ["PYTHONHASHSEED"] = "0"
rn.seed(0)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
tf.set_random_seed(0)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)


In [11]:
# Baseline --> Assign all observations to = 0 (no crime)

model = MyLSTM(**def_params)

train_scores_df, test_scores_df, y_test_probs = time_series_cv(x_all,
                                                               y_all,
                                                               n_splits=n_splits,
                                                               model=model,
                                                               fit_params=fit_params,
                                                               baseline=True)

---------- split 0 ----------
[16:34:43] train_index:0~3099 test_index:3100~6199
[16:34:44] train_log_loss:3.79146095053 test_log_loss:2.74473264132
[16:34:44] train_acc:0.890225962056 test_acc:0.920531850638

---------- split 1 ----------
[16:34:44] train_index:0~6199 test_index:6200~9299
[16:34:45] train_log_loss:3.2677981798 test_log_loss:2.88087476388
[16:34:45] train_acc:0.905387552169 test_acc:0.916590132466

---------- split 2 ----------
[16:34:45] train_index:0~9299 test_index:9300~12399
[16:34:48] train_log_loss:3.13886193424 test_log_loss:3.13772728226
[16:34:48] train_acc:0.909120638834 test_acc:0.909153490373

---------- split 3 ----------
[16:34:48] train_index:0~12399 test_index:12400~15499
[16:34:50] train_log_loss:3.13869660008 test_log_loss:3.45396337131
[16:34:50] train_acc:0.909125425748 test_acc:0.89999751781

---------- split 4 ----------
[16:34:50] train_index:0~15499 test_index:15500~18599
[16:34:53] train_log_loss:3.20163402345 test_log_loss:3.63217123786
[16:34

In [12]:
# train metrics for baseline
train_scores_df.to_csv("./results/train_scores_base.csv", index=False)
train_scores_df

,acc,log_loss
0,0.890226,3.791461
1,0.905388,3.267798
2,0.909121,3.138862
3,0.909125,3.138697
4,0.907303,3.201634


In [13]:
# test metrics for baseline
test_scores_df.to_csv("./results/test_scores_base.csv", index=False)
test_scores_df

,acc,log_loss
0,0.920532,2.744733
1,0.916590,2.880875
2,0.909153,3.137727
3,0.899998,3.453963
4,0.894838,3.632171


In [14]:
# Actual Model --> Assign all observations to prediction

# define model
model = MyLSTM(**def_params)

# time series cross validation
train_scores_df, test_scores_df, y_test_probs = time_series_cv(x_all,
                                                               y_all,
                                                               n_splits=n_splits,
                                                               model=model,
                                                               fit_params=fit_params,
                                                               baseline=False)

---------- split 0 ----------
[16:34:53] train_index:0~3099 test_index:3100~6199
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 2789 samples, validate on 310 samples
Epoch 1/2
2789/2789 [==============================] - 9s 3ms/step - loss: 0.7049 - acc: 0.5343 - val_loss: 0.6954 - val_acc: 0.5656
Epoch 2/2
2789/2789 [==============================] - 7s 2ms/step - loss: 0.7000 - acc: 0.5417 - val_loss: 0.6920 - val_acc: 0.5705
[16:35:19] train_log_loss:0.695406697066 test_log_loss:0.694134724343
[16:35:19] train_acc:0.556556705637 test_acc:0.561718006636

---------- split 1 ----------
[16:35:19] train_index:0~6199 test_index:6200~9299
Train on 5579 samples, validate on 620 samples
Epoch 1/2
5579/5579 [==============================] - 15s 3ms/step - loss: 0.7185 - acc: 0.4905 - val_loss: 0.7071 - val_acc: 0.5181
Epoch 2/2
5579/5579 [=============================

# Result

In [15]:
# train metrics
train_scores_df.to_csv("./results/train_scores.csv", index=False)
train_scores_df

,acc,log_loss
0,0.556557,0.695407
1,0.530701,0.699029
2,0.574985,0.672406
3,0.644711,0.660270
4,0.691637,0.630809


In [16]:
# test metrics
test_scores_df.to_csv("./results/test_scores.csv", index=False)
test_scores_df

,acc,log_loss
0,0.561718,0.694135
1,0.532405,0.698288
2,0.575436,0.672231
3,0.639090,0.661259
4,0.686365,0.633362


In [17]:
# predicted probability of crime occurance
y_probs_df = pd.DataFrame(y_test_probs, index=y_datetime, columns=geo_ids)
y_probs_df = y_probs_df.loc[(y_probs_df != 0).any(axis=1), :]
y_probs_df.to_csv("./results/y_probs.csv", index=True)
y_probs_df.head()

,06075010100,06075010200,06075010300,06075010400,06075010500,06075010600,06075010700,06075010800,06075010900,06075011000,06075011100,06075011200,06075011300,06075011700,06075011800,06075011901,06075011902,06075012000,06075012100,06075012201,06075012202,06075012301,06075012302,06075012401,06075012402,06075012501,06075012502,06075012601,06075012602,06075012700,06075012800,06075012901,06075012902,06075013000,06075013101,06075013102,06075013200,06075013300,06075013400,06075013500,06075015100,06075015200,06075015300,06075015400,06075015500,06075015600,06075015700,06075015801,06075015802,06075015900,06075016000,06075016100,06075016200,06075016300,06075016400,06075016500,06075016600,06075016700,06075016801,06075016802,06075016900,06075017000,06075017101,06075017102,06075017601,06075017700,06075017801,06075017802,06075017902,06075018000,06075020100,06075020200,06075020300,06075020401,06075020402,06075020500,06075020600,06075020700,06075020800,06075020900,06075021000,06075021100,06075021200,06075021300,06075021400,06075021500,06075021600,06075021700,06075021800,06075022600,06075022702,06075022704,06075022801,06075022802,06075022803,06075022901,06075022902,06075022903,06075023001,06075023003,06075023102,06075023103,06075023200,06075023300,06075023400,06075025100,06075025200,06075025300,06075025401,06075025402,06075025403,06075025500,06075025600,06075025701,06075025702,06075025800,06075025900,06075026001,06075026002,06075026003,06075026004,06075026100,06075026200,06075026301,06075026302,06075026303,06075026401,06075026402,06075026403,06075026404,06075030101,06075030102,06075030201,06075030202,06075030301,06075030302,06075030400,06075030500,06075030600,06075030700,06075030800,06075030900,06075031000,06075031100,06075031201,06075031202,06075031301,06075031302,06075031400,06075032601,06075032602,06075032700,06075032801,06075032802,06075032901,06075032902,06075033000,06075033100,06075033201,06075033203,06075033204,06075035100,06075035201,06075035202,06075035300,06075035400,06075040100,06075040200,06075042601,06075042602,06075042700,06075042800,06075045100,06075045200,06075047600,06075047701,06075047702,06075047801,06075047802,06075047901,06075047902,06075060100,06075060400,06075060502,06075060700,06075061000,06075061100,06075061200,06075061400,06075061500,06075980200,06075980300,06075980501,06075980600,06075980900
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-09-17 08:00:00,0.530684,0.572423,0.592704,0.555982,0.475939,0.405431,0.386603,0.466787,0.42318,0.433274,0.631907,0.637933,0.486153,0.656319,0.434664,0.447131,0.693416,0.542471,0.463125,0.633612,0.454787,0.634261,0.465814,0.366476,0.42897,0.363859,0.450052,0.529857,0.354626,0.44623,0.424036,0.59413,0.555709,0.565789,0.65038,0.557958,0.500365,0.385756,0.472088,0.43866,0.599187,0.51435,0.380129,0.603535,0.333337,0.618085,0.517336,0.406854,0.621121,0.412891,0.400021,0.420851,0.370045,0.422213,0.411828,0.492077,0.364683,0.451872,0.56592,0.465557,0.499741,0.463858,0.48647,0.386243,0.466711,0.483786,0.423118,0.508436,0.518269,0.470124,0.526725,0.555131,0.445206,0.621487,0.443921,0.544684,0.525822,0.479753,0.619346,0.653866,0.50232,0.526141,0.513674,0.404537,0.38474,0.54509,0.458631,0.39089,0.578679,0.473942,0.483772,0.692104,0.504677,0.357368,0.510099,0.534341,0.427666,0.484478,0.471532,0.539134,0.619797,0.468579,0.616577,0.568478,0.283186,0.356241,0.561634,0.473525,0.573286,0.389897,0.374763,0.381538,0.496699,0.555098,0.580991,0.440208,0.46526,0.473323,0.537275,0.371779,0.504821,0.463777,0.406748,0.446597,0.422183,0.434971,0.365635,0.572779,0.438708,0.492859,0.630715,0.604769,0.397748,0.530038,0.577851,0.612307,0.441779,0.657327,0.594043,0.562528,0.482752,0.440091,0.487053,0.4635,0.516052,0.629469,0.637401,0.419892,0.552388,0.518645,0.328704,0.572443,0.590698,0.454097,0.418051,0.43164,0.476611,0.524849,0.520106,0.746402,0.462905,0.5